# Finding midpoints by intersecting diffusions

In [1]:
import phate
import numpy as np
import scprep

The old wise programmers say to "take the simple things first". Let's follow this adage. 

In [2]:
# the algorithm
def find_diffusion_midpoint(i,j,P,threshold=0.01):
    """
    Diffuses at increasing scales until a point of non-negligible overlap appears between the dirac diffused from i and the dirac diffused from j. Returns this (these) point(s) as the diffusion midpoints.
    """
    assert i != j
    # initialize diracs at i and j
    diffused_i = np.zeros(P.shape[0])
    diffused_i[i] = 1
    diffused_j = np.zeros(P.shape[0])
    diffused_j[j] = 1
    indices_of_intersection = []
    while len(indices_of_intersection) == 0:
        # take one step of diffusion
        diffused_i = diffused_i @ P
        diffused_j = diffused_j @ P
        # find the intersection as the product of the diffusions
        intersection = diffused_i * diffused_j
        print(max(intersection),len(intersection.nonzero()[0]))
        # remove points in the intersection which fall beneath the threshold
        intersection = (intersection >= threshold).astype(int) * intersection
        indices_of_intersection = intersection.nonzero()[0]
    return indices_of_intersection

In [3]:
a = np.random.rand(8)
a

array([0.49249829, 0.30506222, 0.99881745, 0.55308346, 0.85450986,
       0.61329652, 0.10752496, 0.62414169])

In [4]:
((a>0.3).astype(int)*a).nonzero()[0]

array([0, 1, 2, 3, 4, 5, 7])

# Experiments on the Planar graph

In [5]:
grid = np.mgrid[0:4:0.05,0:4:0.05].T.reshape(-1,2)

In [ ]:
from phate import phate
phate_op = phate.PHATE(decay=1, n_landmark=8000)
phate_coords = phate_op.fit_transform(grid)

In [ ]:
# Test that PHATE worked as expected
scprep.plot.scatter2d(phate_op.embedding)

Looks pretty good to me. I think PHATE has recovered all of the information in this graph, which bodes well for the diffusion operator PHATE used.

In [ ]:
G = phate_op.graph
P = phate_op.diff_op

In [ ]:
P.shape

A complication: PHATE has approximated the diffusion operator using "landmarks," resulting in a matrix which is too small to apply 

It's time to test our method for el primero tiempo! 

In [ ]:
i = 1400
j = 4800
midpoints = find_diffusion_midpoint(i,j, P, threshold=0)
midpoints

In [ ]:
distribution = np.zeros(len(grid))
distribution[midpoints]=1
distribution[i] = 2
distribution[j] = 2
scprep.plot.scatter2d(grid,c=distribution)

In [ ]:
G.K